# Recurrent NNs for sequence modeling

## 1. Why sequence models?

Models like Recurrent neural networks (RNNs) have significantly contributed in areas like Natural Language Processing and speech recognition. Some examples:

- speech recognition: given an input audio file, and the output is a sequence of words.
- Music generation: Output y is a sequence, x can be nothing or the first few notes or...
- sentiment classification: is the review good or bad? Or how many stars?
- Machine translation: google translate
- ...

Let's look at this example. Imagine we have a dataset where each observation is a sentence, and for each sentence we want a model to detect which words are associated with locations. Let's look at the sentence: 

"While we were in school, mom went shopping in the square."

This is a sentence with 11 words, or put differently, a sequence with 11 elements. If this is the first sentence in our entire input space, we would denote it $ x^{(1)}= (x^{(1)<1>},...,x^{(1)<11>})$.

$y^{(1)}$ would then be: $(0,0,0,0,1,0,0,0,0,0,1)$, with 11 elements, $(y^{(1)<1>},...,y^{(1)<11>})$.

The sentence has eleven words, $T_x^{(1)} = T_y^{(1)} = 11$. As $T_x$ and $T_y$ will differ depending on the sentence length, they need the subscript (i) too!
Note that, this just represents this particular sentence $x^{(1)}$. Other sentences will have other lengths. 

## 2. RNN model

We have done NLP before, looking at the data sets with bank complaints. Why are "regular' neural networks sometimes insufficient to deal with NLP and sequence-type problems?
- We created vectors with 0s and 1s denoting if certain words are in a given bank complaint, but no information on the sequence of the word is used!
- In this example, inputs and outputs can be different lengths for different index numbers (i)
- General neural networks cannot learn features across different positions of text.

Take first word $x^{<1>}$, feed it in an NN layer, and try to predict $\hat y^{<1>}$. previous words also have an effect on the output for words later in the sequence. We use the weights $w_{ax}, w_{aa}$ and $w_{ya}$

![title](RNN_2.png)

Disadvantage: networks only uses words earlier in the sequence. Eg:

In "Europe, people tend to use square meters instead of square feet."

The "meters" and "feet" would have been useful to identify that in this case, square is not a physical location.

$a^{<0>}$ = vector of zeros

$a^{<1>} = g(w_{aa}a^{<0>} +w_{ax}x^{<1>}+b_a )$, tanh or relu

$\hat y^{<1>} = g(w_{ya}a^{<1>} + b_y )$, sigmoid

simpler notation:

$a^{<1>} = g(w_{a}[a^{<0>},x^{<1>}]+b_a )$

$\hat y^{<t>} = g(w_{y}a^{<t>}+b_y )$

Matrix $w_a=[w_{aa}; w_{ax}]$



"backpropagation through time" --> Loss function in each vertical in the image above, and then take the sum over all of them, also, right-to=left backpropagation

## 3. Different types of architectures

- Many-to-many --> (location identifyer)
- Many-to-one --> text classifyer (good vs bad review)


![title](RNN_manytoone.png)

- One-to-many --> music generation

![title](RNN_onetomany.png)

- many to many, but input and output lengths are different

![title](RNN_manytomany.png)

Sources:

https://www.coursera.org/learn/nlp-sequence-models/lecture/gw1Xw/language-model-and-sequence-generation

https://www.coursera.org/learn/nlp-sequence-models/lecture/MACos/sampling-novel-sequences


## 4. Issues in sequence models

- Vanishing gradients: plural subject, and maybe much later in the sentence we'll need a plural verb.
- Exploding gradients: you'll notice NaNs, solution is gradient clipping (but, exploding gradients are rare in recurrent networks).

### 4.1 Solution for vanishing gradients: Gated Recurrent Unit (GRU)

Add a memory cell C. A simplified GRU then looks like this:

IN a GRU, $a^{<t>} = C^{<t>}$
Next, candidate for replacing $C^{<t>}$, which is 
    
$\tilde C ^{<t>} = \tanh(w_c[C^{<t-1>}, x^{<t>}]+b_c)$

$\Gamma_u = \sigma(w_u[c^{<t-1>}, x^{<t>}]+b_u])$
--> always 0 or 1

$C ^{<t>} =\Gamma_u * \tilde C ^{<t>} + (1- \Gamma_u)* C^{<t-1>}$

The $\Gamma$ is used to decide whether or not to update! If update gate is 0, $C^{<t>}$ will not be updated!

Depending on how many hidden activation values you have, $C^{<t>}, \tilde C^{<t>}$ and $\Gamma_u$ will be vectors of values, and the last equation above is 

A FULL GRU adds another gate which tells us how important $C^{<t-1>}$ for the update of $\tilde C ^{<t>}$. The new equations are:

$\tilde C ^{<t>} = \tanh(w_c[\Gamma_r * C^{<t-1>}, x^{<t>}]+b_c)$

$\Gamma_u = \sigma(w_u[c^{<t-1>}, x^{<t>}]+b_u])$

$C ^{<t>} =\Gamma_u * \tilde C ^{<t>} + (1- \Gamma_u)* C^{<t-1>}$

$\Gamma_r = \sigma(w_r[c^{<t-1>}, x^{<t>}]+b_r])$


https://www.coursera.org/learn/nlp-sequence-models/lecture/agZiL/gated-recurrent-unit-gru

### 4.2 Solution for vanishing gradients: Long Short Term Memory (LSTM)

In LSTM, we do not use that $a^{<t>}=C ^{<t>}$

$\tilde C ^{<t>} = \tanh(w_c[a^{<t-1>}, x^{<t>}]+b_c)$

$\Gamma_u = \sigma(w_u[a^{<t-1>}, x^{<t>}]+b_u])$

$\Gamma_f = \sigma(w_u[c^{<t-1>}, x^{<t>}]+b_f])$

$\Gamma_o = \sigma(w_o[c^{<t-1>}, x^{<t>}]+b_o])$

$\tilde C ^{<t>} = \Gamma_u* \tilde C ^{<t>} + \Gamma_f* C ^{<t-1>}$ 
$a ^{<t>} = \Gamma_o* \tanh C ^{<t>} $

With $\Gamma_u, \Gamma_f$ and $\Gamma_o$ the update, forget and output gate respectively.

### 4.3 Which one to use?

GRU: simpler, and easier to run on a big network

LSTM: more powerful, more flexible

LSTM is generally considered as the default, but more complex.

### 4.4 Other networks 

Bi-directional RNN: take information about the entire sequence, not just what is preceding!

Deep RNNs: several vertical layers

## 5. Word embeddings

### 5.1 One-hot representations.

- As we've seen before, you use the 10,000 most recurring words in the training set, or look at an online dictionary of commonly used words.
- Apply dictionary to our text to create one-hot representations per word
- each of the vector is 9,999 times 0 and 1 time 1.
- With 11 words in a sentence, 11 one-hot vectors of 10,000 units.
- If word not in in your 10,000 words dictionary, you create a new vector

--> each word is a thing to itself, and cannot easily generalize across words. eg, relationship between fruits, between locations,... unclear

Solution: word embeddings! Example:

|      | Duke  | Dutchess | cow | dog |
| -----|------ |------- |-----| ----|
|Noble |  1    | 1      | 0.03 |  0.01 |
|  food | 0.01 | -0.02 | 0.7  | 0.08 |
| Gender | -1  |  1   | 0.02 | 0.03 |

The words on the left column are "features". Now imagine "duke", "dutchess", "cow" and "dog" represent locations in our 10,000 most recurreing words vector. If Duke is on the 1265th place, we can create a n-dimensional vector (with n the amount of features) representing the word "duke". This way, our algorithm will know that "cow" and "dog" are more similar than "cow" and "duke".

Word embeddings can also be visualized, using the t-SNE algorithnm to make an n-dimensional space 2D!

"While we were in school, mom went shopping in the square."

--> if we know that school and square are physical locations, similar words like street, class,... would be recognized more easily. 

How do transfer learning from word embeddings? 
- 1) learn word embeddings for large body of text (or download pre-trained embeddings online)
- 2) Transfer embedding to smaller training set
- 3) If you want, you can continue finetuning the word embeddings with new data.

word embeddings especially useful when you are doing tasks on small data sets.

### 5.2 Properties of word embeddings


1. by taking differences between word embeddings, you can get a better understanding of words that are related to each other. eg. vector difference between boy and girl is similar to the vector difference between duke and dutchess! Only difference is gender. A high degree of similarity between 

arg max $sim(e_{w}, e_{boy}-e_{duke}+e_{duchess})$, usually cosine similarity is used.

|n|   a   | ...  | cow | duke | ... | zucchini|
|--|-----|------|------- |-----| ----|-----|
|1 | gender|... | 0.03 | -1 |  0.01 | -0.02|
|2 |food | ... | -0.02 | 0.7  | 0.08 |0.99|
|... | ... |... | ... | ... | ... | ... | 
|500| furniture |...  | 0.02 | 0.01| ...|-0.03|

when you multiply one-hot vector representing "cow" with embeddingsmatrix, you will basically just keep the column with the embedding representation for cow! In practice, you'll use an embedding layer that simply pulls out a column, because performing these multiplications is much slower when we have huge vectors with all 0's and just one 1.

### 5.3 Some examples of language models

#### 5.3.1 learning word embeddings

You want to predict the next word in a sequence

1. A sentence: get the word embedding vector for each word. Embedding vector has a fixed amount of rows (=number of embeddings).

2. Feed all of them into a neural  network. Usual practice is to fix the amount of presceding words to, let's say, 5. If embedding has 500 rows, leads to 2500-dimensional feature vector going into the first layer. End of the network is a softmax.

other possibilities: last 3 words and 3 words after, last 1 word, nearby 1 word.

#### 5.3.2 word2vec

simpler way to learn embeddings: **skip-gram model**. Context-target pairs. You randomly pick a word to be context word, and then randomly pick a target word withing a specified window from the context word, let's say, 5 words apart.

"The teacher was writing on the blackboard while the principal walked in"

Context: blackboard 

target: could be principal, while, blackboard,...

We want to map our context "c" (blackboard) to our target "t"

The model looks like this (with e the embessing : softmax: 
$P(t|c) = \displaystyle \frac{e^{\Theta^T_te_c}}{\sum^{10,000}_{j=1}e^{\Theta^T_je_c}}$

loss function: $\mathcal L (\hat y, y ) = -\sum^{10,000}_{i=1}y_i \log \hat y_i$

$\Theta_t$ is the parameter associated with output t

https://www.coursera.org/learn/nlp-sequence-models/lecture/8CZiw/word2vec

Problem: slow, so hierarchical softmax is suggested!

#### 5.3.3 Negative sampling

word2vec is slow! Try this:

- given a pair of words, try to predict whether or not it is a context target pair (1 or 0). This way, label word combinations with 0 or 1. First actively label pairs of words, then create a supervised learning problem.
- logistic regression model: P(y=1|c,t) with c and t the context target pair.
- Do a small logistic regression model with about 5-6 content-target pairs (only 1 1, rest is 0 so you deliberately use your negative samples!). Repeat this 10,000 times where each time another content word is chosen. faster than working with a softmax classifier! 


#### 5.3.4 Negative sampling

GloVe: less frequently used, but has enthusiast because so simple!

Again c,t: use x_{i,j} to define how often 2 words appear close to each other.

# 6. Words embeddings applications